# **BERTopic - Tutorial**
We start with installing bertopic from pypi before preparing the data. 

**NOTE**: Make sure to select a GPU runtime. Otherwise, the model can take quite some time to create the document embeddings!

# **Prepare data**
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts on 20 topics.

In [32]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

# **Create Topics**
We select the "english" as the main language for our documents. If you want a multilingual model that supports 50+ languages, please select "multilingual" instead. 

In [33]:
model = BERTopic(language="english")
topics, probs = model.fit_transform(docs)

We can then extract most frequent topics:

In [35]:
model.get_topic_freq().head(5)

,Topic,Count
0,-1,6224
1,0,1833
2,1,586
3,2,526
4,3,480


-1 refers to all outliers and should typically be ignored. Next, let's take a look at the most frequent topic that was generated:

In [34]:
model.get_topic(0)[:10]

[('game', 0.010436520264926533),
 ('team', 0.009084781216657336),
 ('games', 0.007234962878545154),
 ('he', 0.007131864903744603),
 ('players', 0.006351685439415695),
 ('season', 0.006282132628187317),
 ('hockey', 0.006157176094578879),
 ('play', 0.005817036121709545),
 ('25', 0.005687199271496157),
 ('year', 0.0056711166453823885)]

Note that the model is stocastich which mmeans that the topics might differ across runs. 

For a full list of support languages, see the values below:

In [8]:
from bertopic.backend import languages
print(languages)

['afrikaans', 'albanian', 'amharic', 'arabic', 'armenian', 'assamese', 'azerbaijani', 'basque', 'belarusian', 'bengali', 'bengali romanize', 'bosnian', 'breton', 'bulgarian', 'burmese', 'burmese zawgyi font', 'catalan', 'chinese (simplified)', 'chinese (traditional)', 'croatian', 'czech', 'danish', 'dutch', 'english', 'esperanto', 'estonian', 'filipino', 'finnish', 'french', 'galician', 'georgian', 'german', 'greek', 'gujarati', 'hausa', 'hebrew', 'hindi', 'hindi romanize', 'hungarian', 'icelandic', 'indonesian', 'irish', 'italian', 'japanese', 'javanese', 'kannada', 'kazakh', 'khmer', 'korean', 'kurdish (kurmanji)', 'kyrgyz', 'lao', 'latin', 'latvian', 'lithuanian', 'macedonian', 'malagasy', 'malay', 'malayalam', 'marathi', 'mongolian', 'nepali', 'norwegian', 'oriya', 'oromo', 'pashto', 'persian', 'polish', 'portuguese', 'punjabi', 'romanian', 'russian', 'sanskrit', 'scottish gaelic', 'serbian', 'sindhi', 'sinhala', 'slovak', 'slovenian', 'somali', 'spanish', 'sundanese', 'swahili', '

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [19]:
model.topics_[:10]

[0, -1, -1, -1, 30, -1, -1, 0, 0, -1]

# **Embedding model**
You can select any model from `sentence-transformers` and use it instead of the preselected models by simply passing the model through  
BERTopic with `embedding_model`:

In [ ]:
# st_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  


# **Visualize Topics**
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [1]:
model.visualize_topics()

# **Topic Reduction**
Finally, we can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [11]:
model.reduce_topics(docs, nr_topics=60)

# **Topic Representation**
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stop_words or you want to try out a different n_gram_range. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [13]:
model.update_topics(docs, n_gram_range=(1, 3))

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "gpu". Then, we extract the most similar topic and check the results: 

In [24]:
similar_topics, similarity = model.find_topics("gpu", top_n=5); similar_topics

[18, 21, 24, 28, 36]

In [25]:
model.get_topic(18)

[('monitor', 0.02604954056000538),
 ('the', 0.011592478015427971),
 ('monitors', 0.011386430026081376),
 ('vga', 0.0104858000492717),
 ('video', 0.008950171344439876),
 ('with', 0.008770262226959034),
 ('to', 0.008627947931402424),
 ('it', 0.008211951060402312),
 ('and', 0.00815957039585335),
 ('horizontal', 0.007798501042289411)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
# Save model
model.save("my_model")

In [ ]:
# Load model
my_model = BERTopic.load("my_model")